## Playground for RSA for Hanabi with 'intention representation' ##

In [2]:
import numpy as np
import utility_saskia as ut
# needs the utility.py file from:
# https://github.com/saskiabruhn/Hanabi/blob/master/hanabi_learning_environment/agents/utility.py

For this examples, let's imagine a 2 player game of Hanabi.

Situation: Player 2 has a red card on chop position that Player 1 wants her "to save". Player 1 gives the hint "red card (only) on 1st position" to Player 2. Player 2 then reasons about Player 1's intention behind this hint. For now, this means she only reasons about the 1st card in her hand.


In [3]:
# list of all available actions

# TO DO: make list of all possible actions compatible to Saskia's utility function
all_actions = [{'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'R'},
              {'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'B'},
              {'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'G'},
              {'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'Y'},
              {'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'W'},
              {'action_type': 'DISCARD'}]
               
all_intentions = ['play', 'save', 'discard']

# current situation, taken from Saskia's utility function
estimated_board = {'life_tokens': 3,
                   'information_tokens': 7,
                   'fireworks': {'R': 3, 'Y': 2, 'G': 4, 'W': 1, 'B': 2},
                   'discard_pile': ['B2', 'B3', 'B2'],
                   'estimated_hands': [
                       [{'color': None, 'rank': -1},
                        {'color': None, 'rank': 0},
                        {'color': None, 'rank': -1},
                        {'color': None, 'rank': -1},
                        {'color': None, 'rank': -1}],
                       [{'color': 'B', 'rank': 2},
                        {'color': 'R', 'rank': 0},
                        {'color': 'Y', 'rank': 4},
                        {'color': 'B', 'rank': 3},
                        {'color': 'Y', 'rank': 0}]
                    ]
                   }


# my old abitrary fixed utilities
#utilities = {
#    'to play': {'red': 0.4, 'blue': -3, 'discard': -1},
#    'to save': {'red': 0.9, 'blue': -2, 'discard': -3},
#    'to discard': {'red': -1, 'blue': 0.01, 'discard': 0.9}
#}

# assuming flat priors for now
priors = {
    'play': 1,
    'save': 1,
    'discard': 1
}



Pragmatic Listener 

formular: probability_to_play = likelihood_action_given_intention * prior[intention]) / sum (all other likelihoods * prioir[intention])

In [4]:
def prag_listener(action, estimated_board, intention):
    bayes_numerator = (prag_speaker(action, estimated_board, intention) * priors[intention])
    bayes_denominator = 0
    for specific_intention in all_intentions:
        bayes_denominator += prag_speaker(action, estimated_board, specific_intention) * priors[intention]
    return bayes_numerator / bayes_denominator


Pragmatic speaker

In [11]:
def prag_speaker(action, estimated_board, intention):
    alpha = 1
    softmax_numerator = np.exp(alpha * ut.utility(intention, estimated_board, card))
    softmax_denominator = 0
    for specific_action in all_actions:
        # TO DO:
        # PROBLEM: HOW TO INCLUDE DIFFERENT ACTIONS HERE?
        softmax_denominator += np.exp(alpha * ut.utility(intention, estimated_board, card))
    return softmax_numerator / softmax_denominator

## Experiments ##

In [9]:
last_action = {'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'R'}
card = {'color': 'R', 'rank': 2}

prob_to_play = prag_listener(last_action, estimated_board, 'play')
prob_to_save = prag_listener(last_action, estimated_board, 'save')
prob_to_discard = prag_listener(last_action, estimated_board, 'discard')

print("Hint is: 'red'\n")
print("probability to save: ",prob_to_save)
print("probability to play: ",prob_to_play)
print("probability to discard: ",prob_to_discard)

Hint is: 'red'

probability to save:  0.3333333333333333
probability to play:  0.3333333333333333
probability to discard:  0.33333333333333337
